In [122]:

from natsort import natsorted
from skimage.morphology import binary_erosion

In [180]:
import sys
sys.path.insert(1, '../SyMBac/')

import numpy as np
from scipy.ndimage.morphology import distance_transform_edt
from PIL import Image
import os
from csbdeep.utils import normalize
#sys.path.insert(0,'../../../../SyMBac')
from skimage.measure import label
from tqdm.notebook import tqdm
import tifffile
from glob import glob
from joblib import Parallel, delayed
def unet_weight_map(y, wc=None, w0 = 10, sigma = 5):

    """
    Generate weight maps as specified in the U-Net paper
    for boolean mask.

    "U-Net: Convolutional Networks for Biomedical Image Segmentation"
    https://arxiv.org/pdf/1505.04597.pdf

    Parameters
    ----------
    mask: Numpy array
        2D array of shape (image_height, image_width) representing binary mask
        of objects.
    wc: dict
        Dictionary of weight classes.
    w0: int
        Border weight parameter.
    sigma: int
        Border width parameter.

    Returns
    -------
    Numpy array
        Training weights. A 2D array of shape (image_height, image_width).
    """

    labels = label(y)
    no_labels = labels == 0
    label_ids = sorted(np.unique(labels))[1:]

    if len(label_ids) > 1:
        distances = np.zeros((y.shape[0], y.shape[1], len(label_ids)))

        for i, label_id in enumerate(label_ids):
            distances[:,:,i] = distance_transform_edt(labels != label_id)

        distances = np.sort(distances, axis=2)
        d1 = distances[:,:,0]
        d2 = distances[:,:,1]
        w = w0 * np.exp(-1/2*((d1 + d2) / sigma)**2) * no_labels
    else:
        w = np.zeros_like(y)
    if wc:
        class_weights = np.zeros_like(y)
        for k, v in wc.items():
            class_weights[y == k] = v
        w = w + class_weights
    return w
wc = {
    0: 1, # background
    1: 5  # objects
}


def preprocess_convolutions(x, name):
    name = name.split("/")[-1].split(".")[0]+".tiff"
    x = normalize(x, 1,99.8, axis=(0,1))
    tifffile.imwrite(base_directory + "CROPPED_FILTERED/"+name, x)

def preprocess_masks(x, name):
    name = name.split("/")[-1].split(".")[0]+".tiff"
    x = x.astype(bool)
    tifffile.imwrite(base_directory + "CROPPED_MASKS/"+name,x)
    # .astype(np.uint8)*255

def preprocess_weightmaks(x, name):
    name = name.split("/")[-1].split(".")[0]+".tiff"
    x = unet_weight_map(x, wc)
    image = x.astype(np.uint8)
    tifffile.imwrite(base_directory + "WEIGHTMAPS/"+name,x)

#DelTA does not allow touching cells, therefore erode all cells, and use these new masks for training, segmentation, and quantification of accuracy. 
def correct_masks_for_DeLTA(masks):
    mask_labels = np.unique(masks)[1:]
    eroded_masks = np.zeros(masks.shape, dtype=bool)
    for mask_label in mask_labels:
        eroded_masks += binary_erosion(masks == mask_label).astype(bool)
    return eroded_masks

In [185]:
for data_dir in tqdm(data_dirs):
    print(data_dir)
    data_dirs = glob("data/*")
    data_type = "training_data"
    mask_dirs = natsorted(glob(f"{data_dir}/{data_type}/*masks.png"))
    image_dirs = natsorted(glob(f"{data_dir}/{data_type}/*[!masks].png"))


    masks = [np.array(Image.open(mask_dir)) for mask_dir in mask_dirs]
    corrected_masks = [np.pad(correct_masks_for_DeLTA(mask), 16) for mask in tqdm(masks)] #pad images by 16 on each axis for a final shape of 832 (fits into the U-net)
    corrected_images = [np.pad(np.array(Image.open(image_dir)), 16) for image_dir in tqdm(image_dirs)]

    try:
        os.mkdir(f"{data_dir}/DeLTA")
    except:
        pass
    base_directory = f"{data_dir}/DeLTA/preprocessed/"
    try:
        os.mkdir(base_directory)
    except:
        pass
    try: 
        os.mkdir(base_directory + "/CROPPED_FILTERED")
        os.mkdir(base_directory + "/CROPPED_MASKS")
        os.mkdir(base_directory + "/WEIGHTMAPS")
    except:
        pass


    _ = Parallel(n_jobs=-1)(delayed(preprocess_convolutions)(x, name) for x, name in tqdm(zip(corrected_images,image_dirs)))

    _ = Parallel(n_jobs=-1)(delayed(preprocess_masks)(x, name) for x, name in tqdm(zip(corrected_masks,image_dirs)))

    _ = Parallel(n_jobs=-1)(delayed(preprocess_weightmaks)(x, name) for x, name in tqdm(zip(corrected_masks,image_dirs)))

  0%|          | 0/6 [00:00<?, ?it/s]

data/5_10


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [186]:
3

3